In [4]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
import numpy as np
from astropy.constants import c, G, M_sun
from scipy.integrate import dblquad
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)  # 假设一个平坦的宇宙模型



In [5]:
def critical_density(cosmo, zd, zs):
    # 计算 D_LS
    D_L  = cosmo.angular_diameter_distance(zd).to(u.m)
    D_S  = cosmo.angular_diameter_distance(zs).to(u.m)
    D_LS = cosmo.angular_diameter_distance_z1z2(zd,zs).to(u.m)
    # 计算临界密度
    sigma_crit = (c**2 / (4 * np.pi * G)) * (D_S / (D_L * D_LS))

    # 返回单位为 kg/m²
    return sigma_crit.to(u.kg / u.m**2)

def Mlensing(thetaE_phy, Sigma_crit):
    
    # 计算 M_lensing
    Mlensing = np.pi * (thetaE_phy ** 2) * Sigma_crit
    
    return Mlensing

def einstein_radius_to_physical_meters(cosmo, thetaE_arcsec, zd):
    """
    将爱因斯坦半径从角秒转换为红移 z_d 处的物理尺度（单位为米）。
    
    参数：
    cosmo : astropy.cosmology.Cosmology
        用于计算角直径距离的宇宙学模型（例如 FlatLambdaCDM）。
    thetaE_arcsec : float
        爱因斯坦半径，单位为角秒（arcsec）。
    zd : float
        透镜天体的红移 z_d。
    
    返回：
    float
        对应物理尺度，单位为米（m）。
    """
    
    # 将角秒转换为弧度
    thetaE_rad = thetaE_arcsec * u.arcsec.to(u.rad)
    
    # 计算角直径距离 D_A(z_d) (单位为 Mpc)
    D_A = cosmo.angular_diameter_distance(zd).to(u.m)  # 转换为米
    
    # 计算物理尺度
    r_phys = D_A * thetaE_rad  # 物理尺度，单位为米（m）
    
    return r_phys

def thetaE_2_Mlensing(thetaE,Sigma_crit):
    thetaE_phy = einstein_radius_to_physical_meters(cosmo, thetaE, zd)
    return Mlensing(thetaE_phy,Sigma_crit)

In [6]:
# # 邓力艋透镜
# zd=0.397
# zs=1.4792

# Sigma_crit = critical_density(cosmo, zd, zs)
# print(f"临界密度: {Sigma_crit}")

# thetaE1 = 0.424  # 角秒
# Ml1 = thetaE_2_Mlensing(thetaE1, Sigma_crit)
# print(f"透镜质量: {Ml1}")
# Ml1 / M_sun

In [7]:
zd = 0.79  # 透镜天体红移
zs = 2.16  # 源天体红移

Sigma_crit = critical_density(cosmo, zd, zs)
print(f"临界密度: {Sigma_crit}")

thetaE1 = 0.852  # 角秒
Ml1 = thetaE_2_Mlensing(thetaE1, Sigma_crit)
print(f"透镜质量: {Ml1}")
Ml1 / M_sun

临界密度: 4.604545663295443 kg / m2
透镜质量: 5.587910658395708e+41 kg


<Quantity 2.81024086e+11>

In [8]:
# 示例使用：
thetaE2 = 0.539  # 角秒
Ml2 = thetaE_2_Mlensing(thetaE2, Sigma_crit)

print(f"透镜质量: {Ml2}")
Ml2 / M_sun

透镜质量: 2.2363913029653777e+41 kg


<Quantity 1.12471344e+11>

In [9]:
def total_mass(
    Sigma_crit,  # kg/m2
    qEPL,        # 无量纲
    gammaEPL,    # 无量纲
    thetaE,      # 角秒
    qSER,        # 无量纲
    phiSER,      # 度
    re,          # 角秒
    zd,          # 星系红移
    cosmo        # astropy 宇宙学模型
):
    # 1. 角直径距离 (m)
    D_A = cosmo.angular_diameter_distance(zd).to(u.m).value

    # 2. 单位转换因子：arcsec -> rad
    arcsec2rad = np.pi / (180.0 * 3600.0)

    # 3. 椭圆半轴 (arcsec)
    a = np.sqrt(re / qSER)
    b = np.sqrt(re * qSER)

    # 4. 旋转角度 (rad)
    phi = np.deg2rad(phiSER)

    # 5. Σ(x,y) 函数，输入 x,y 单位为 arcsec
    def Sigma_xy(x, y):
        r_term = np.sqrt(qEPL * x**2 + y**2 / qEPL)
        return Sigma_crit * (3.0 - gammaEPL) / 2.0 * (thetaE / r_term)**(gammaEPL - 1.0)

    # 6. 极坐标下的被积函数 (r, θ)
    def integrand(theta, r):
        # 归一化 u,v
        u_ = r * np.cos(theta)
        v_ = r * np.sin(theta)
        # 拉伸到 x', y'
        x_p = a * u_
        y_p = b * v_
        # 旋转到原坐标 x, y
        x =  x_p * np.cos(phi) - y_p * np.sin(phi)
        y =  x_p * np.sin(phi) + y_p * np.cos(phi)
        # 返回 Σ(x,y) * r
        return Sigma_xy(x, y) * r

    # 7. 数值积分：r ∈ [0,1], θ ∈ [0,2π]
    integral, error = dblquad(
        integrand,
        0.0,        # r 下限
        1.0,        # r 上限
        lambda r: 0.0,        # θ 下限
        lambda r: 2*np.pi     # θ 上限
    )

    # 8. 转换到物理面积并乘以 a·b
    area_factor = (D_A * arcsec2rad)**2
    mass = integral * a * b * area_factor  # 单位：kg

    return mass


In [10]:

Mlens1 = total_mass(
    Sigma_crit=Sigma_crit.value,    # kg/m2
    qEPL=0.695,
    gammaEPL=1.75,
    thetaE=0.852,       # arcsec
    qSER=0.86,
    phiSER=-89.459,       # deg
    re=(5.29*0.168)*2,         # arcsec (a·b = re)
    zd=zd,
    cosmo=cosmo
)

Mlens2 = total_mass(
    Sigma_crit=Sigma_crit.value,    # kg/m2
    qEPL=0.605,
    gammaEPL=1.84,
    thetaE=0.539,        # arcsec
    qSER=0.6359,
    phiSER=82.74,       # deg
    re=(2.277*0.168)**2,            # arcsec (a·b = re)
    zd=zd,
    cosmo=cosmo
)

Mlens1=Mlens1*u.kg
Mlens2=Mlens2*u.kg
print(f"Total mass of lens1 = {Mlens1:.3e} ")
print(f"Total mass of lens2 = {Mlens2:.3e} ")

Mlens1 / M_sun

Total mass of lens1 = 9.630e+41 kg 
Total mass of lens2 = 1.424e+41 kg 


<Quantity 4.8431005e+11>

In [11]:
Mlens2 / M_sun

<Quantity 7.16243999e+10>

In [12]:
(Mlens1+Mlens2)/ M_sun

<Quantity 5.55934449e+11>

In [13]:
1-(2.7233583942094177/2 * 10**11 *M_sun)/Mlens1

<Quantity 0.71884143>

In [14]:
1-(1.1052682195316391/2 * 10**11 *M_sun)/Mlens2

<Quantity 0.22842759>